In [ ]:
import os
import numpy as np
import pandas as pd

In [ ]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.layers import Dropout
from tensorflow.python.keras.layers import Activation
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [ ]:
monthly_sales = pd.read_csv('https://raw.githubusercontent.com/yuetongzhang/Costco_Yuetong-Zhang/main/Costco_Yuetong%20Zhang/Costco_Monthly%20Sales%20from%202012%20to%202021.csv')
monthly_sales1=monthly_sales.drop(columns=['Growth Rate','Date'])
monthly_sales1=monthly_sales1.dropna()
monthly_sales_df=pd.DataFrame(monthly_sales1)
monthly_sales_df = monthly_sales_df.rename(columns={'Net Sales (billions)': 'Total Sales'})
monthly_sales_df = monthly_sales_df[['Total Sales', 'Year', 'Month']]
monthly_sales_df.head()

,Total Sales,Year,Month
0,7.00,2012,1
2,9.13,2012,3
3,7.25,2012,4
4,7.67,2012,5
5,9.18,2012,6


In [ ]:
url='https://raw.githubusercontent.com/kzhao6/tripods-sales/main/Boarded_Passengers%3ACar_Sale.csv'
flyers = pd.read_csv(url)
#flyers = pd.read_csv('https://raw.githubusercontent.com/kzhao6/tripods-sales/main/Boarded_Passengers_since_2003_-_Copy-3.csv')
dom_flyers = flyers.drop(columns=['INTERNATIONAL','TOTAL'])
dom_flyers=dom_flyers.dropna()
dom_flyers_df=pd.DataFrame(dom_flyers)
#dom_flyers_df = monthly_sales_df.rename(columns={'Net Sales (billions)': 'Total Sales'})
monthly_sales_df = monthly_sales_df[['Total Sales', 'Year', 'Month']]
dom_flyers_df.head()

,Year,Month,DOMESTIC,New,Used,Total Sales New,Total Sales Used
0,2012,3,38479741.0,28280.0,62426.0,790798381.0,513936240.0
1,2012,4,36956112.0,27467.0,56913.0,766243639.0,493881213.0
2,2012,5,38245610.0,29390.0,55096.0,822158205.0,489914146.0
3,2012,6,39823763.0,29662.0,53148.0,847168622.0,487154088.0
4,2012,7,41112651.0,26593.0,52146.0,791818346.0,468241971.0


In [ ]:
k=dom_flyers_df

In [ ]:
real_rate=pd.read_csv('https://raw.githubusercontent.com/RamenMode/NeuralNetsSales/main/100-EmploymentRateReal.csv')
df_real_rate=pd.DataFrame(real_rate)
df_real_rate.head()
irate= df_real_rate.iloc[:,3]
monthly_sales_df['Inflation Rate']=irate
print(monthly_sales_df)

     Total Sales  Year  Month  Inflation Rate
0           7.00  2012      1             2.9
2           9.13  2012      3             2.3
3           7.25  2012      4             1.7
4           7.67  2012      5             1.7
5           9.18  2012      6             1.4
..           ...   ...    ...             ...
109        14.05  2021      2             2.6
110        18.21  2021      3             4.2
111        15.21  2021      4             5.0
112        15.59  2021      5             5.4
113        18.92  2021      6             NaN

[113 rows x 4 columns]


In [ ]:
cpi= pd.read_csv('https://raw.githubusercontent.com/kzhao6/tripods-sales/main/historical_US_CPI.csv')
df_cpi = pd.DataFrame(cpi)
df_cpi.head()
cpi_value = df_cpi.iloc[:,1]
k['CPI']=cpi_value
print(cpi_value)

0      228.807
1      229.187
2      228.713
3      228.524
4      228.590
        ...   
108    264.793
109    266.832
110    268.551
111    270.981
112    272.265
Name: Value, Length: 113, dtype: float64


In [ ]:
Income_tax=pd.read_csv('https://raw.githubusercontent.com/kzhao6/tripods-sales/main/US_Income_Tax_Rate.csv')
df_tax=pd.DataFrame(Income_tax)
df_tax.head()
tax = df_tax.iloc[:,1]
monthly_sales_df['Income Tax']=tax

In [ ]:
no_employment= pd.read_csv('https://raw.githubusercontent.com/RamenMode/NeuralNetsSales/main/100.csv')
df_no_employment = pd.DataFrame(no_employment)
df_no_employment.head()
no_job = df_no_employment.iloc[:,7]
monthly_sales_df['100-Unemployment']=no_job


In [ ]:
def remove_outliers(df, name, sd):
    drop_rows = df.index[(np.abs(df[name] - df[name].mean())
                          >= (sd * df[name].std()))]
    df.drop(drop_rows, axis=0, inplace=True)

In [ ]:
remove_outliers(monthly_sales_df,'CPI',monthly_sales_df['CPI'].std())

In [ ]:
len(monthly_sales_df)
len(cpi_value)
len(tax)
len(no_job)

883

In [ ]:
ycol='Used'
data_2020 = k.loc[k['Year'] == 2020]
Y_2020 = data_2020[ycol]
X_2020 = data_2020[['Month','Year','CPI']]

In [ ]:
data_before = k.loc[k['Year'] < 2020]
Y_before = data_before[ycol]
X_before = data_before[['Month','Year','CPI']]

In [ ]:
XTrain, XTest, YTrain, YTest = train_test_split(X_before, Y_before, test_size = 0.2, shuffle = True)
#XTrain = X_before
#YTrain = Y_before

model0 = Sequential()
model0.add(Dense(450, activation = 'relu', input_dim = 3))
model0.add(Dense(200, activation = 'relu'))
model0.add(Dropout(0.1))
model0.add(Dense(100, activation = 'relu'))
model0.add(Dropout(0.2))
model0.add(Dense(1, activation = 'linear'))
model0.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
loss30=0
loss2020=0
for i in range(30):
  model0.fit(XTrain, YTrain, epochs=100, batch_size=100)
  loss1 = model0.evaluate(XTest, YTest)
  loss2 = model0.evaluate(X_2020, Y_2020)
  loss30+=loss1
  loss2020+=loss2
  
  
  

In [ ]:
print(np.sqrt(loss30/30))

8094.905649028076


In [ ]:
#loss = model0.evaluate(XTest, YTest)
print(np.sqrt(loss30/30),'loss before 2020')
print(k[ycol].std(),'std of',ycol)
predict_2020 = model0.predict(X_2020)
r2_2020 = r2_score(Y_2020, predict_2020)

8094.905649028076 loss before 2020
8071.138819271415 std of Used


In [ ]:
XTest = X_2020
YTest = Y_2020
#loss2020 = model0.evaluate(XTest, YTest)
print(np.sqrt(loss2020/30),'loss 2020')
print(k[ycol].std(),'std of',ycol)

12840.43845564992 loss 2020
8071.138819271415 std of Used
